## DATASCI W205 - An introduction to Apache	Spark and Spark SQL (Lab 6)

In [1]:
!echo $SPARK_HOME

/usr/lib/spark/


In [2]:
cd /usr/lib/spark/bin

/usr/lib/spark/bin


In [3]:
x = [1,2,3,4,5,6,7,8,9,10]

In [4]:
distdata = sc.parallelize(x)

In [5]:
distdata.count()

10

### Step 2: Load a File	and	Count the Rows

In [6]:
ls

beeline*               pyspark*      spark-shell*   utils.sh*
compute-classpath.sh*  run-example*  spark-sql*
load-spark-env.sh*     spark-class*  spark-submit*


In [13]:
pwd

u'/root/ipython-nbs'

In [7]:
cd /usr/local/

/usr/local


In [2]:
crimefile = sc.textFile("/usr/local/Crimes_-_2001_to_present.csv")

In [3]:
crimefile

/usr/local/Crimes_-_2001_to_present.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [16]:
!wc -c Crimes_-_2001_to_present.csv

1376617753 Crimes_-_2001_to_present.csv


In [21]:
crimefile.count()

5862796

In [22]:
crimefile.take(10)

[u'ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location',
 u'10184515,HY372204,08/06/2015 11:55:00 PM,033XX W DIVERSEY AVE,2027,NARCOTICS,POSS: CRACK,STREET,true,false,1412,014,35,22,18,1153440,1918377,2015,08/13/2015 12:57:42 PM,41.931870591,-87.711546895,"(41.931870591, -87.711546895)"',
 u'10184607,HY372206,08/06/2015 11:55:00 PM,035XX S RHODES AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0212,002,4,35,08B,1180132,1881331,2015,08/13/2015 12:57:42 PM,41.82964147,-87.614598779,"(41.82964147, -87.614598779)"',
 u'10190430,HY374464,08/06/2015 11:50:00 PM,047XX W HARRISON ST,0430,BATTERY,AGGRAVATED: OTHER DANG WEAPON,GAS STATION,false,false,1131,011,24,25,04B,1144626,1896881,2015,08/13/2015 12:57:42 PM,41.873054046,-87.744479572,"(41.873054046, -87.744479572)"',
 u'10185476,HY372534,08/06/2015 11:50:00 PM,030XX W FLETCHE

### Step 3.	Filter Records and	Structures

In [4]:
def remove_header(itr_index, itr): return iter(list(itr)[1:]) if itr_index == 0 else itr 

In [5]:
noHeaderCrimeData2 = crimefile.mapPartitionsWithIndex(remove_header)
##noHeaderCrimeData2.count()


In [6]:
narcoticsCrimes = noHeaderCrimeData2.filter(lambda x: "NARCOTICS" in x)

In [29]:
narcoticsCrimes.count()

663712

In [30]:
narcoticsCrimes.take(20)

[u'10184515,HY372204,08/06/2015 11:55:00 PM,033XX W DIVERSEY AVE,2027,NARCOTICS,POSS: CRACK,STREET,true,false,1412,014,35,22,18,1153440,1918377,2015,08/13/2015 12:57:42 PM,41.931870591,-87.711546895,"(41.931870591, -87.711546895)"',
 u'10184524,HY372200,08/06/2015 11:40:00 PM,002XX W 88TH ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,true,false,0634,006,21,49,18,1175866,1846548,2015,08/13/2015 12:57:42 PM,41.7342901,-87.631292466,"(41.7342901, -87.631292466)"',
 u'10184554,HY372217,08/06/2015 11:38:00 PM,078XX S PAULINA ST,2028,NARCOTICS,POSS: SYNTHETIC DRUGS,STREET,true,false,0611,006,17,71,18,1166342,1852418,2015,08/13/2015 12:57:42 PM,41.750606117,-87.666017303,"(41.750606117, -87.666017303)"',
 u'10184503,HY372179,08/06/2015 11:35:00 PM,019XX N MOZART ST,2022,NARCOTICS,POSS: COCAINE,STREET,true,false,1421,014,35,22,18,1157157,1912657,2015,08/13/2015 12:57:42 PM,41.916099694,-87.698043163,"(41.916099694, -87.698043163)"',
 u'10184527,HY372190,08/06/2015 11:32:00 PM,015XX W H

In [7]:
 narcoticsCrimeRecords = narcoticsCrimes.map(lambda r : r.split(","))

In [8]:
narcoticsCrimeRecords.first()

[u'10184515',
 u'HY372204',
 u'08/06/2015 11:55:00 PM',
 u'033XX W DIVERSEY AVE',
 u'2027',
 u'NARCOTICS',
 u'POSS: CRACK',
 u'STREET',
 u'true',
 u'false',
 u'1412',
 u'014',
 u'35',
 u'22',
 u'18',
 u'1153440',
 u'1918377',
 u'2015',
 u'08/13/2015 12:57:42 PM',
 u'41.931870591',
 u'-87.711546895',
 u'"(41.931870591',
 u' -87.711546895)"']

In [12]:
 narcoticsCrimeRecords.count()

663712

### Step 4.	Key Values

In [16]:
##original, keeps the key in the tuple
narcoticsCrimeTuples = narcoticsCrimes.map(lambda x:(x.split(",")[0], x))

##original, exclude the key from second value in the tuple
narcoticsCrimeTuplesfixed = narcoticsCrimes.map(lambda x:(x.split(",")[0],  ', '.join(x.split(",")[1:])))


In [18]:
print narcoticsCrimeTuplesfixed.take(1)

[(u'10184515', u'HY372204, 08/06/2015 11:55:00 PM, 033XX W DIVERSEY AVE, 2027, NARCOTICS, POSS: CRACK, STREET, true, false, 1412, 014, 35, 22, 18, 1153440, 1918377, 2015, 08/13/2015 12:57:42 PM, 41.931870591, -87.711546895, "(41.931870591,  -87.711546895)"')]


In [20]:
firstTuple=narcoticsCrimeTuples.first()

In [21]:
len(firstTuple)

2

In [23]:
firstTuple[1]
##firstTuple[1]

u'10184515,HY372204,08/06/2015 11:55:00 PM,033XX W DIVERSEY AVE,2027,NARCOTICS,POSS: CRACK,STREET,true,false,1412,014,35,22,18,1153440,1918377,2015,08/13/2015 12:57:42 PM,41.931870591,-87.711546895,"(41.931870591, -87.711546895)"'

In [24]:
sorted_tuple = narcoticsCrimeTuplesfixed.sortByKey()

##sorted.first()
##%reset_selective narcoticsCrimeRecords
##narcoticsCrimeRecords.unpersist()

#### Submission 1

In [25]:
sorted_tuple.first()

(u'10000014',
 u'HY189846, 03/18/2015 05:52:00 PM, 107XX S EGGLESTON AVE, 1822, NARCOTICS, MANU/DEL:CANNABIS OVER 10 GMS, RESIDENCE, true, false, 2233, 022, 34, 49, 18, 1175162, 1833372, 2015, 03/25/2015 12:42:30 PM, 41.698149046, -87.634263523, "(41.698149046,  -87.634263523)"')

In [26]:
narcoticsCrimeTuplesfixed.first()

(u'10184515',
 u'HY372204, 08/06/2015 11:55:00 PM, 033XX W DIVERSEY AVE, 2027, NARCOTICS, POSS: CRACK, STREET, true, false, 1412, 014, 35, 22, 18, 1153440, 1918377, 2015, 08/13/2015 12:57:42 PM, 41.931870591, -87.711546895, "(41.931870591,  -87.711546895)"')

In [ ]:
##You could change your code to as I did earlier
narcoticsCrimeTuplesfixed = narcoticsCrimes.map(lambda x:(x.split(",")[0],  ', '.join(x.split(",")[1:])))


#### Attempt to connect to Hive

In [27]:
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)
!which hive
sqlContext.sql("SHOW TABLES") ##not connecte to hive
##probably have to follow steps here https://getblueshift.com/connecting-hive-and-spark-on-aws-in-five-easy-steps/
sqlContext.sql("show databases")

###Step 7 Accessing	Spark SQL in Python Code

In [33]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *

In [34]:
sqlContext = SQLContext(sc)

In [35]:
ls

DATASCI W205 - An introduction to Apache?Spark and Spark SQL - lab 6 - Greg Ceccarelli.ipynb
derby.log
metastore_db/
week6-class.ipynb


In [36]:
ls /usr/local/

Anaconda-2.3.0-Linux-x86_64.sh   derby.log            lib/
Crimes_-_2001_to_present.csv     etc/                 lib64/
Python-2.7.6/                    files/               libexec/
Python-2.7.6.tar.xz              games/               sbin/
bin/                             include/             share/
certificates/                    ipythonpass.txt      src/
cloudera-manager-installer.bin*  kafka_2.10-0.8.2.1/  w205-labs-exercises/


In [37]:
 lines = sc.textFile('/usr/local/w205-labs-exercises/data/weblog_lab.csv')

In [38]:
parts = lines.map(lambda l: l.split('\t'))

In [40]:
Web_Session_Log = parts.map(lambda p: (p[0], p[1],p[2],
p[3],p[4]))

In [41]:
Web_Session_Log

PythonRDD[30] at RDD at PythonRDD.scala:42

In [42]:
Web_Session_Log.first()

(u'date', u'userid', u'sessionid', u'productid', u'refererurl')

In [43]:
 schemaString = 'DATETIME USERID SESSIONID PRODUCTID REFERERURL'

In [44]:
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]

In [45]:
fields

[StructField(DATETIME,StringType,true),
 StructField(USERID,StringType,true),
 StructField(SESSIONID,StringType,true),
 StructField(PRODUCTID,StringType,true),
 StructField(REFERERURL,StringType,true)]

In [47]:
schema = StructType(fields)

In [48]:
schema

StructType(List(StructField(DATETIME,StringType,true),StructField(USERID,StringType,true),StructField(SESSIONID,StringType,true),StructField(PRODUCTID,StringType,true),StructField(REFERERURL,StringType,true)))

In [50]:
 schemaWebData = sqlContext.createDataFrame(Web_Session_Log, schema)

In [52]:
schemaWebData.registerTempTable('Web_Session_Log')

In [53]:
 results = sqlContext.sql('SELECT count(*) FROM Web_Session_Log')

####Submission 2 & 3

In [59]:
results.head()
results.show()
##sqlContext.sql('SELECT count(*) FROM Web_Session_Log')

c0   
40002


In [60]:
 resultsEbay = sqlContext.sql('select count(*) from Web_Session_Log where REFERERURL = "http://www.ebay.com"')

In [61]:
resultsEbay.show()

c0  
3943
